<h1>Final project</h1>

<h2>Simple CNN</h2>

In [ ]:
# Encode the labels
X = df['image']
y = df['label']

encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Convert the labels to one-hot encoding
y = tf.keras.utils.to_categorical(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)), #defines the first convolutional layer of the model. The layer has 32 filters of size 3x3, uses ReLU activation function and expects input images of size 64x64 with 3 color channels.
    tf.keras.layers.MaxPooling2D(2, 2), #adds a max pooling layer with a pool size of 2x2. The layer reduces the spatial dimensions of the output from the previous layer by a factor of 2.
    tf.keras.layers.Dropout(0.25), #randomly drops out 25% of the input units during training to reduce overfitting.
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), #defines the second convolutional layer of the model. The layer has 64 filters of size 3x3 and uses ReLU activation function.
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(), # flattens the output from the previous layer into a 1D array.
    tf.keras.layers.Dense(128, activation='relu'), #adds a fully connected layer with 128 neurons and ReLU activation function.
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(encoder.classes_), activation='softmax') #adds the output layer of the model with the number of neurons equal to the number of unique labels in the dataset. The layer uses softmax activation function to output a probability distribution over the labels.
])

# Compile the model
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model
model.fit(X_train.to_list(), y_train, epochs=10, validation_data=(X_val.to_list(), y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_val.to_list(), y_val)
print(f"Validation loss: {loss:.4f}")
print(f"Validation accuracy: {accuracy:.4f}")


<h2>GoogleNet CNN</h2>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, AveragePooling2D, Flatten, Dense

# Define the input shape of the image as a tuple of (64, 64, 3) 
input_shape = (64, 64, 3)

# Define the input tensor
inputs = Input(shape=input_shape)

# First block
x = Conv2D(64, (7,7), strides=(2,2), padding='same', activation='relu')(inputs) #convolutional layer with 64 filters of size (7,7), using stride of 2 and padding same.
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x) #max-pooling layer with pool size (3,3), stride 2, and padding same.
x = BatchNormalization()(x)

# Second block
x = Conv2D(64, (1,1), activation='relu')(x) # 1x1 convolution with 64 filters and ReLU activation.
x = Conv2D(192, (3,3), padding='same', activation='relu')(x) #3x3 convolution with 192 filters, padding same and ReLU activation.
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x) #max-pooling with pool size (3,3), stride 2, and padding same
x = BatchNormalization()(x)

# Inception blocks
def inception_module(x, filters): #Defines a function called "inception_module" that takes two inputs, a tensor and a list of filters.
    path1 = Conv2D(filters[0], (1,1), activation='relu')(x) #convolutional layer with the first filter size from the filter list, a kernel size of (1,1), and the ReLU activation function to the input tensor.
    path1 = Conv2D(filters[1], (3,3), padding='same', activation='relu')(path1) #Adds another convolutional layer with the second filter size from the filter list, a kernel size of (3,3), the same padding, and the ReLU activation function to the previous layer.

    path2 = Conv2D(filters[2], (1,1), activation='relu')(x) #Adds another convolutional layer with the third filter size from the filter list, a kernel size of (1,1), and the ReLU activation function to the input tensor.
    path2 = Conv2D(filters[3], (5,5), padding='same', activation='relu')(path2) #Adds another convolutional layer with the fourth filter size from the filter list, a kernel size of (5,5), the same padding, and the ReLU activation function to the previous layer.

    path3 = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(x)
    path3 = Conv2D(filters[4], (1,1), activation='relu')(path3) #applies a 1x1 convolution with the number of filters specified in filters[4] to the output of the max pooling layer path

    output = concatenate([path1, path2, path3], axis=-1) #creates an output tensor with a larger number of channels, which contains the outputs of all three paths.

    return output

#Applies multiple instances of the inception_module function, using different combinations of filter sizes and numbers of channels, to build a deep neural network with multiple Inception modules.
x = inception_module(x, [64, 96, 128, 16, 32])
x = inception_module(x, [128, 128, 192, 32, 96])
#output tensor goes through another max pooling layer with a pool size of 3x3, stride of 2x2, and padding of 'same'. This downsamples the tensor by a factor of 2 in each spatial dimension.
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

x = inception_module(x, [192, 96, 208, 16, 48])
x = inception_module(x, [160, 112, 224, 24, 64])
x = inception_module(x, [128, 128, 256, 24, 64])
x = inception_module(x, [112, 144, 288, 32, 64])
x = inception_module(x, [256, 160, 320, 32, 128])
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

# Final block
x = AveragePooling2D(pool_size=(7,7))(x)
x = Dropout(0.4)(x)

x = Flatten()(x)
x = Dense(1000, activation='relu')(x)
x = Dropout(0.4)(x)
outputs = Dense(10, activation='softmax')(x)

# Define the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation loss: {loss:.
